In [1]:
import openml
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

from automl import AutoML

In [4]:
import openml
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

from automl import AutoML


# Инициализация переменных
task_id = 359962

task = None
dataset = None
X = None
y = None
categorical = None
feature_names = None
n_folds = 3
class_labels = None
label_mapping = None
data_loaded = False

print("[INFO] Начало загрузки данных из OpenML...")

try:
    print(f"[INFO] Загрузка задачи из OpenML (task_id={task_id})...")
    task = openml.tasks.get_task(task_id)
    print(f"[INFO] Задача загружена: {task.task_id}")
    
    print("[INFO] Загрузка датасета...")
    dataset = task.get_dataset()
    print(f"[INFO] Датасет загружен: {dataset.name}")
    
    print("[INFO] Получение параметров задачи...")
    params = task.estimation_parameters
    n_folds = int(params.get("number_folds", 3)) - 1
    print(f"[INFO] Количество фолдов: {n_folds}")
    
    class_labels = task.class_labels
    print(f"[INFO] Классы: {class_labels}")
    
    print("[INFO] Загрузка данных...")
    X, y, categorical, feature_names = dataset.get_data(target=dataset.default_target_attribute)
    print(f"[INFO] Данные загружены. Размер X: {X.shape}, размер y: {y.shape}")
    
    # Проверки данных
    # if X is None:
    #     print("[ERROR] X равен None. Пропуск дальнейшей обработки.")
    # elif isinstance(X, pd.DataFrame) and X.empty:
    #     print("[ERROR] X пустой DataFrame. Пропуск дальнейшей обработки.")
    # elif isinstance(X, np.ndarray) and X.size == 0:
    #     print("[ERROR] X пустой ndarray. Пропуск дальнейшей обработки.")
    # elif len(X) == 0:
    #     print("[ERROR] X имеет нулевую длину. Пропуск дальнейшей обработки.")
    # elif y is None:
    #     print("[ERROR] y равен None. Пропуск дальнейшей обработки.")
    # elif isinstance(y, pd.Series) and y.empty:
    #     print("[ERROR] y пустой Series. Пропуск дальнейшей обработки.")
    # elif isinstance(y, (pd.Series, np.ndarray)) and len(y) == 0:
    #     print("[ERROR] y имеет нулевую длину. Пропуск дальнейшей обработки.")
    # elif len(X) != len(y):
    #     print(f"[ERROR] Несоответствие размеров: X={len(X)}, y={len(y)}. Пропуск дальнейшей обработки.")
    # else:
    #     print("[INFO] Создание маппинга меток...")
    #     label_mapping = {v: k for k, v in enumerate(class_labels)}
    #     if isinstance(y, pd.Series):
    #         y = np.array(y.map(label_mapping), dtype=int)
    #     else:
    #         # Если y уже массив или другой тип, преобразуем через pandas для совместимости
    #         y_series = pd.Series(y)
    #         y = np.array(y_series.map(label_mapping), dtype=int)
        
    #     # Проверка после преобразования
    #     if len(y) == 0:
    #         print("[ERROR] y пустой после преобразования. Пропуск дальнейшей обработки.")
    #     elif len(np.unique(y)) < 2:
    #         print(f"[ERROR] Недостаточно классов в y: {np.unique(y)}. Пропуск дальнейшей обработки.")
    #     else:
    #         print(f"[INFO] Уникальные классы в y: {np.unique(y)}")
    #         print(f"[INFO] Распределение классов: {pd.Series(y).value_counts().to_dict()}")
            
    #         data_loaded = True
    #         print("[INFO] Данные успешно загружены и подготовлены")
    
except Exception as e:
    print(f"[ERROR] Ошибка при загрузке данных: {type(e).__name__}: {e}")
    data_loaded = False
    print("[WARNING] Пайплайн остановлен из-за ошибки загрузки данных")

[INFO] Начало загрузки данных из OpenML...
[INFO] Загрузка задачи из OpenML (task_id=359962)...
[INFO] Задача загружена: 359962
[INFO] Загрузка датасета...
[INFO] Датасет загружен: kc1
[INFO] Получение параметров задачи...
[INFO] Количество фолдов: 9
[INFO] Классы: ['false', 'true']
[INFO] Загрузка данных...
[INFO] Данные загружены. Размер X: (2109, 21), размер y: (2109,)


In [7]:
{v: k for k, v in enumerate(class_labels)}

{'false': 0, 'true': 1}

In [5]:

y

0       False
1        True
2        True
3        True
4        True
        ...  
2104    False
2105    False
2106    False
2107    False
2108    False
Name: defects, Length: 2109, dtype: bool

In [3]:
automl = None
model_trained = False

if not data_loaded:
    print("[ERROR] Данные не загружены. Пропуск обучения модели.")
else:
    try:
        # Проверки перед обучением
        if X is None or y is None:
            print("[ERROR] X или y не определены. Пропуск обучения модели.")
        elif not isinstance(X, (pd.DataFrame, np.ndarray)):
            print(f"[ERROR] X должен быть DataFrame или ndarray, получен {type(X)}. Пропуск обучения модели.")
        elif not isinstance(y, np.ndarray):
            print(f"[ERROR] y должен быть ndarray, получен {type(y)}. Пропуск обучения модели.")
        else:
            print("[INFO] Инициализация AutoML...")
            print(f"[INFO] Параметры: task=classification, n_splits={n_folds}, n_jobs=3")
            
            automl = AutoML(
                task='classification',
                use_preprocessing_pipeline=True,
                feature_selector_type=None,
                use_val_test_pipeline=True,
                auto_models_init_kwargs = {
                    "metric": "roc_auc",
                    "time_series": False,
                    "models_list": ["all",],
                    "blend": True,
                    "stack": True,
                    "n_splits": n_folds,
                },
                n_jobs=16, 
                random_state=0,
            )
            print("[INFO] AutoML инициализирован")
            
            print("[INFO] Начало обучения модели...")
            automl = automl.fit(
                X, y, 
                auto_model_fit_kwargs = {
                    "tuning_timeout": 10
                }
            )
            
            # Проверка результата обучения
            if automl is None:
                print("[ERROR] AutoML вернул None после обучения. Пропуск дальнейшей обработки.")
            elif not hasattr(automl, 'auto_model'):
                print("[ERROR] AutoML не имеет атрибута auto_model. Пропуск дальнейшей обработки.")
            else:
                model_trained = True
                print("[INFO] Модель успешно обучена")
                
                if hasattr(automl.auto_model, 'best_score'):
                    print(f"[INFO] Лучший OOF score: {automl.auto_model.best_score}")
        
    except Exception as e:
        print(f"[ERROR] Ошибка при обучении модели: {type(e).__name__}: {e}")
        import traceback
        print(f"[ERROR] Трассировка: {traceback.format_exc()}")
        model_trained = False
        print("[WARNING] Модель не обучена")

[INFO] Инициализация AutoML...
[INFO] Параметры: task=classification, n_splits=9, n_jobs=3
[2025-11-15 01:06:32,664] - [  PREPROC   ] - Успешно заданы шаги pipeline
[2025-11-15 01:06:32,664] - [ VAL TESTS  ] - Успешно заданы шаги pipeline
[INFO] AutoML инициализирован
[INFO] Начало обучения модели...
[2025-11-15 01:06:35,213] - [Pipeline] .. (step 1 of 6) Processing nan_cols_dropper, total=   0.0
[2025-11-15 01:06:35,219] - [Pipeline] ....... (step 2 of 6) Processing nan_imputer, total=   0.0
[2025-11-15 01:06:35,223] - [Pipeline] .... (step 3 of 6) Processing qconst_dropper, total=   0.0
[2025-11-15 01:06:35,226] - [  PREPROC   ] - Corr features to drop: ['V3', 'V3']
[2025-11-15 01:06:35,227] - [Pipeline] . (step 4 of 6) Processing corr_cols_dropper, total=   0.0
[2025-11-15 01:06:35,235] - [Pipeline] .... (step 5 of 6) Processing outlier_capper, total=   0.0
[2025-11-15 01:06:35,239] - [Pipeline] ... (step 6 of 6) Processing feature_encoder, total=   0.0
[    FIT     ] - X_test is No

In [4]:
oof_score = None
score = None
predictions = None

if not model_trained or automl is None:
    print("[ERROR] Модель не обучена. Пропуск предсказаний и оценки.")
else:
    try:
        print("[INFO] Получение предсказаний...")
        
        # Проверки перед предсказанием
        if not hasattr(automl, 'predict'):
            print("[ERROR] AutoML не имеет метода predict. Пропуск предсказаний.")
        elif X is None:
            print("[ERROR] X не определен. Пропуск предсказаний.")
        else:
            predictions = automl.predict(X)
            
            if predictions is None:
                print("[ERROR] Предсказания равны None. Пропуск дальнейшей обработки.")
            elif len(predictions) == 0:
                print("[ERROR] Предсказания пустые. Пропуск дальнейшей обработки.")
            else:
                print(f"[INFO] Получены предсказания. Размер: {predictions.shape}")
                
                # Проверка формы предсказаний для бинарной классификации
                if len(predictions.shape) == 2 and predictions.shape[1] >= 2:
                    predictions_proba = predictions[:, 1]
                    print("[INFO] Используется вероятность второго класса")
                elif len(predictions.shape) == 1:
                    predictions_proba = predictions
                    print("[INFO] Используются предсказания напрямую")
                else:
                    print(f"[ERROR] Неожиданная форма предсказаний: {predictions.shape}. Пропуск дальнейшей обработки.")
                    predictions_proba = None
                
                if predictions_proba is not None:
                    # Получение OOF score
                    if hasattr(automl.auto_model, 'best_score'):
                        oof_score = automl.auto_model.best_score
                        print(f"[INFO] OOF score: {oof_score}")
                    else:
                        print("[WARNING] best_score не найден в auto_model")
                    
                    # Вычисление метрики на полных данных
                    print("[INFO] Вычисление ROC-AUC на полных данных...")
                    if y is None:
                        print("[ERROR] y не определен. Пропуск вычисления метрики.")
                    elif len(y) != len(predictions_proba):
                        print(f"[ERROR] Несоответствие размеров: y={len(y)}, predictions={len(predictions_proba)}. Пропуск вычисления метрики.")
                    else:
                        try:
                            score = roc_auc_score(y, predictions_proba)
                            print(f"[INFO] ROC-AUC на полных данных: {score}")
                            print(f"[INFO] Результаты: OOF={oof_score}, Full={score}")
                        except Exception as metric_error:
                            print(f"[ERROR] Ошибка при вычислении ROC-AUC: {type(metric_error).__name__}: {metric_error}")
                            print("[WARNING] Метрика не вычислена")
        
    except Exception as e:
        print(f"[ERROR] Ошибка при получении предсказаний или оценке: {type(e).__name__}: {e}")
        import traceback
        print(f"[ERROR] Трассировка: {traceback.format_exc()}")
        print("[WARNING] Предсказания или оценка не выполнены")

# Вывод результатов
if oof_score is not None and score is not None:
    print(f"\n[RESULT] OOF score: {oof_score}, Full data score: {score}")
    (oof_score, score)
else:
    print("\n[RESULT] Результаты недоступны")
    None

[INFO] Получение предсказаний...
[INFO] Получены предсказания. Размер: (748, 2)
[INFO] Используется вероятность второго класса
[INFO] OOF score: 0.7591563177606938
[INFO] Вычисление ROC-AUC на полных данных...
[INFO] ROC-AUC на полных данных: 0.7710181352257046
[INFO] Результаты: OOF=0.7591563177606938, Full=0.7710181352257046

[RESULT] OOF score: 0.7591563177606938, Full data score: 0.7710181352257046


## Вариант со стратифицированным тестовым набором


In [5]:
# Вариант со стратифицированным тестовым набором
automl_stratified = None
model_stratified_trained = False
X_train_strat = None
X_test_strat = None
y_train_strat = None
y_test_strat = None

if not data_loaded:
    print("[ERROR] Данные не загружены. Пропуск обучения модели со стратифицированным тестом.")
else:
    try:
        # Проверки перед разделением данных
        if X is None or y is None:
            print("[ERROR] X или y не определены. Пропуск разделения данных.")
        elif not isinstance(X, (pd.DataFrame, np.ndarray)):
            print(f"[ERROR] X должен быть DataFrame или ndarray, получен {type(X)}. Пропуск разделения данных.")
        elif not isinstance(y, np.ndarray):
            print(f"[ERROR] y должен быть ndarray, получен {type(y)}. Пропуск разделения данных.")
        else:
            print("[INFO] Разделение данных на стратифицированные train/test...")
            print(f"[INFO] Размер исходных данных: {len(X)}")
            
            # Стратифицированное разделение (20% на тест)
            X_train_strat, X_test_strat, y_train_strat, y_test_strat = train_test_split(
                X, y, 
                test_size=0.2, 
                random_state=42, 
                stratify=y
            )
            
            print(f"[INFO] Train размер: {len(X_train_strat)}, Test размер: {len(X_test_strat)}")
            print(f"[INFO] Распределение классов в train: {pd.Series(y_train_strat).value_counts().to_dict()}")
            print(f"[INFO] Распределение классов в test: {pd.Series(y_test_strat).value_counts().to_dict()}")
            
            print("[INFO] Инициализация AutoML для стратифицированного варианта...")
            print(f"[INFO] Параметры: task=classification, n_splits={n_folds}, n_jobs=16")
            
            automl_stratified = AutoML(
                task='classification',
                use_preprocessing_pipeline=True,
                feature_selector_type=None,
                use_val_test_pipeline=False,
                auto_models_init_kwargs = {
                    "metric": "roc_auc",
                    "time_series": False,
                    "models_list": ["linear", "forests", "boostings"],
                    "blend": True,
                    "stack": True,
                    "n_splits": n_folds,
                },
                n_jobs=16, 
                random_state=0,
            )
            print("[INFO] AutoML инициализирован")
            
            print("[INFO] Начало обучения модели на train данных...")
            automl_stratified = automl_stratified.fit(
                X_train_strat, y_train_strat, 
                auto_model_fit_kwargs = {
                    "tuning_timeout": 10
                }
            )
            
            # Проверка результата обучения
            if automl_stratified is None:
                print("[ERROR] AutoML вернул None после обучения. Пропуск дальнейшей обработки.")
            elif not hasattr(automl_stratified, 'auto_model'):
                print("[ERROR] AutoML не имеет атрибута auto_model. Пропуск дальнейшей обработки.")
            else:
                model_stratified_trained = True
                print("[INFO] Модель успешно обучена на train данных")
                
                if hasattr(automl_stratified.auto_model, 'best_score'):
                    print(f"[INFO] Лучший OOF score на train: {automl_stratified.auto_model.best_score}")
        
    except Exception as e:
        print(f"[ERROR] Ошибка при обучении модели со стратифицированным тестом: {type(e).__name__}: {e}")
        import traceback
        print(f"[ERROR] Трассировка: {traceback.format_exc()}")
        model_stratified_trained = False
        print("[WARNING] Модель не обучена")


[INFO] Разделение данных на стратифицированные train/test...
[INFO] Размер исходных данных: 748
[INFO] Train размер: 598, Test размер: 150
[INFO] Распределение классов в train: {0: 456, 1: 142}
[INFO] Распределение классов в test: {0: 114, 1: 36}
[INFO] Инициализация AutoML для стратифицированного варианта...
[INFO] Параметры: task=classification, n_splits=9, n_jobs=16
[2025-11-15 01:08:19,501] - [  PREPROC   ] - Успешно заданы шаги pipeline


[INFO] AutoML инициализирован
[INFO] Начало обучения модели на train данных...
[2025-11-15 01:08:19,506] - [Pipeline] .. (step 1 of 6) Processing nan_cols_dropper, total=   0.0
[2025-11-15 01:08:19,512] - [Pipeline] ....... (step 2 of 6) Processing nan_imputer, total=   0.0
[2025-11-15 01:08:19,517] - [Pipeline] .... (step 3 of 6) Processing qconst_dropper, total=   0.0
[2025-11-15 01:08:19,520] - [  PREPROC   ] - Corr features to drop: ['V3', 'V3']
[2025-11-15 01:08:19,522] - [Pipeline] . (step 4 of 6) Processing corr_cols_dropper, total=   0.0
[2025-11-15 01:08:19,527] - [Pipeline] .... (step 5 of 6) Processing outlier_capper, total=   0.0
[2025-11-15 01:08:19,532] - [Pipeline] ... (step 6 of 6) Processing feature_encoder, total=   0.0
[2025-11-15 01:08:19,544] - [   MODEL    ] - 1 out of 8. type
[2025-11-15 01:08:19,545] - [   START    ] - Working with type
[2025-11-15 01:08:19,545] - [   START    ] - Tuning type
[2025-11-15 01:08:19,580] - [   OPTUNA   ] - Trial 0. New best score 0

In [6]:
# Оценка на стратифицированном тестовом наборе
oof_score_stratified = None
test_score_stratified = None
train_score_stratified = None
predictions_test_strat = None
predictions_train_strat = None

if not model_stratified_trained or automl_stratified is None:
    print("[ERROR] Модель не обучена. Пропуск предсказаний и оценки.")
else:
    try:
        print("[INFO] Получение предсказаний на тестовом наборе...")
        
        # Проверки перед предсказанием
        if not hasattr(automl_stratified, 'predict'):
            print("[ERROR] AutoML не имеет метода predict. Пропуск предсказаний.")
        elif X_test_strat is None:
            print("[ERROR] X_test_strat не определен. Пропуск предсказаний.")
        else:
            # Предсказания на тестовом наборе
            predictions_test_strat = automl_stratified.predict(X_test_strat)
            
            if predictions_test_strat is None:
                print("[ERROR] Предсказания на тесте равны None. Пропуск дальнейшей обработки.")
            elif len(predictions_test_strat) == 0:
                print("[ERROR] Предсказания на тесте пустые. Пропуск дальнейшей обработки.")
            else:
                print(f"[INFO] Получены предсказания на тесте. Размер: {predictions_test_strat.shape}")
                
                # Проверка формы предсказаний для бинарной классификации
                if len(predictions_test_strat.shape) == 2 and predictions_test_strat.shape[1] >= 2:
                    predictions_test_proba = predictions_test_strat[:, 1]
                    print("[INFO] Используется вероятность второго класса для теста")
                elif len(predictions_test_strat.shape) == 1:
                    predictions_test_proba = predictions_test_strat
                    print("[INFO] Используются предсказания напрямую для теста")
                else:
                    print(f"[ERROR] Неожиданная форма предсказаний на тесте: {predictions_test_strat.shape}. Пропуск дальнейшей обработки.")
                    predictions_test_proba = None
                
                # Предсказания на обучающем наборе (для сравнения)
                print("[INFO] Получение предсказаний на обучающем наборе...")
                predictions_train_strat = automl_stratified.predict(X_train_strat)
                
                if predictions_train_strat is None:
                    print("[WARNING] Предсказания на train равны None.")
                elif len(predictions_train_strat) == 0:
                    print("[WARNING] Предсказания на train пустые.")
                else:
                    print(f"[INFO] Получены предсказания на train. Размер: {predictions_train_strat.shape}")
                    
                    if len(predictions_train_strat.shape) == 2 and predictions_train_strat.shape[1] >= 2:
                        predictions_train_proba = predictions_train_strat[:, 1]
                    elif len(predictions_train_strat.shape) == 1:
                        predictions_train_proba = predictions_train_strat
                    else:
                        predictions_train_proba = None
                
                if predictions_test_proba is not None:
                    # Получение OOF score
                    if hasattr(automl_stratified.auto_model, 'best_score'):
                        oof_score_stratified = automl_stratified.auto_model.best_score
                        print(f"[INFO] OOF score на train: {oof_score_stratified}")
                    else:
                        print("[WARNING] best_score не найден в auto_model")
                    
                    # Вычисление метрики на тестовом наборе
                    print("[INFO] Вычисление ROC-AUC на тестовом наборе...")
                    if y_test_strat is None:
                        print("[ERROR] y_test_strat не определен. Пропуск вычисления метрики.")
                    elif len(y_test_strat) != len(predictions_test_proba):
                        print(f"[ERROR] Несоответствие размеров: y_test={len(y_test_strat)}, predictions={len(predictions_test_proba)}. Пропуск вычисления метрики.")
                    else:
                        try:
                            test_score_stratified = roc_auc_score(y_test_strat, predictions_test_proba)
                            print(f"[INFO] ROC-AUC на тестовом наборе: {test_score_stratified}")
                        except Exception as metric_error:
                            print(f"[ERROR] Ошибка при вычислении ROC-AUC на тесте: {type(metric_error).__name__}: {metric_error}")
                            print("[WARNING] Метрика на тесте не вычислена")
                    
                    # Вычисление метрики на обучающем наборе (для сравнения)
                    if predictions_train_proba is not None and y_train_strat is not None:
                        print("[INFO] Вычисление ROC-AUC на обучающем наборе...")
                        try:
                            train_score_stratified = roc_auc_score(y_train_strat, predictions_train_proba)
                            print(f"[INFO] ROC-AUC на обучающем наборе: {train_score_stratified}")
                        except Exception as metric_error:
                            print(f"[WARNING] Ошибка при вычислении ROC-AUC на train: {type(metric_error).__name__}: {metric_error}")
                    
                    # Вывод результатов
                    print(f"\n[INFO] Результаты стратифицированного варианта:")
                    if oof_score_stratified is not None:
                        print(f"  OOF score (на train): {oof_score_stratified}")
                    if train_score_stratified is not None:
                        print(f"  ROC-AUC на train: {train_score_stratified}")
                    if test_score_stratified is not None:
                        print(f"  ROC-AUC на test: {test_score_stratified}")
        
    except Exception as e:
        print(f"[ERROR] Ошибка при получении предсказаний или оценке: {type(e).__name__}: {e}")
        import traceback
        print(f"[ERROR] Трассировка: {traceback.format_exc()}")
        print("[WARNING] Предсказания или оценка не выполнены")

# Вывод итоговых результатов
if oof_score_stratified is not None and test_score_stratified is not None:
    print(f"\n[RESULT STRATIFIED] OOF score: {oof_score_stratified}, Test score: {test_score_stratified}")
    if train_score_stratified is not None:
        print(f"[RESULT STRATIFIED] Train score: {train_score_stratified}")
    (oof_score_stratified, test_score_stratified, train_score_stratified)
else:
    print("\n[RESULT STRATIFIED] Результаты недоступны")
    None


[INFO] Получение предсказаний на тестовом наборе...
[INFO] Получены предсказания на тесте. Размер: (150, 2)
[INFO] Используется вероятность второго класса для теста
[INFO] Получение предсказаний на обучающем наборе...
[INFO] Получены предсказания на train. Размер: (598, 2)
[INFO] OOF score на train: 0.7514516926118112
[INFO] Вычисление ROC-AUC на тестовом наборе...
[INFO] ROC-AUC на тестовом наборе: 0.8050682261208577
[INFO] Вычисление ROC-AUC на обучающем наборе...
[INFO] ROC-AUC на обучающем наборе: 0.7546716703731158

[INFO] Результаты стратифицированного варианта:
  OOF score (на train): 0.7514516926118112
  ROC-AUC на train: 0.7546716703731158
  ROC-AUC на test: 0.8050682261208577

[RESULT STRATIFIED] OOF score: 0.7514516926118112, Test score: 0.8050682261208577
[RESULT STRATIFIED] Train score: 0.7546716703731158


## Функция для запуска пайплайна



In [7]:
def run_automl_pipeline(
    task_id: int,
    test_size: float = 0.2,
    random_state: int = 42,
    n_jobs: int = 16,
    tuning_timeout: int = 10,
    verbose: bool = True
) -> pd.DataFrame:
    """
    Запускает полный пайплайн AutoML для задачи из OpenML со стратифицированным разделением на train/test.
    
    Параметры:
    ----------
    task_id : int
        ID задачи из OpenML
    test_size : float, default=0.2
        Доля данных для тестового набора
    random_state : int, default=42
        Random state для воспроизводимости
    n_jobs : int, default=16
        Количество параллельных процессов
    tuning_timeout : int, default=10
        Таймаут для тюнинга модели (в секундах)
    verbose : bool, default=True
        Выводить ли информационные сообщения
    
    Возвращает:
    -----------
    pd.DataFrame
        Датафрейм с предсказаниями и метриками:
        - predictions_train: предсказания на обучающем наборе
        - predictions_test: предсказания на тестовом наборе
        - y_true: истинные значения для всех данных
        - y_true_train: истинные значения для train
        - y_true_test: истинные значения для test
        - is_test: флаг, является ли строка тестовой
        - oof_score: OOF score на train
        - train_score: ROC-AUC на train
        - test_score: ROC-AUC на test
        - task_id: ID задачи
        - dataset_name: название датасета
    """
    import traceback
    
    results_dict = {}
    
    try:
        if verbose:
            print(f"[INFO] Начало загрузки данных из OpenML (task_id={task_id})...")
        
        # Загрузка задачи и датасета
        task = openml.tasks.get_task(task_id)
        dataset = task.get_dataset()
        
        if verbose:
            print(f"[INFO] Задача загружена: {task.task_id}")
            print(f"[INFO] Датасет загружен: {dataset.name}")
        
        # Получение параметров
        params = task.estimation_parameters
        n_folds = int(params.get("number_folds", 3)) - 1
        class_labels = task.class_labels
        
        if verbose:
            print(f"[INFO] Количество фолдов: {n_folds}")
            print(f"[INFO] Классы: {class_labels}")
        
        # Загрузка данных
        X, y, categorical, feature_names = dataset.get_data(target=dataset.default_target_attribute)
        
        if verbose:
            print(f"[INFO] Данные загружены. Размер X: {X.shape}, размер y: {y.shape}")
        
        # Проверки данных
        if X is None or (isinstance(X, pd.DataFrame) and X.empty) or (isinstance(X, np.ndarray) and X.size == 0) or len(X) == 0:
            raise ValueError("X пустой или None")
        if y is None or (isinstance(y, pd.Series) and y.empty) or (isinstance(y, np.ndarray) and len(y) == 0) or len(y) == 0:
            raise ValueError("y пустой или None")
        if len(X) != len(y):
            raise ValueError(f"Несоответствие размеров: X={len(X)}, y={len(y)}")
        
        # Преобразование меток
        label_mapping = {v: k for k, v in enumerate(class_labels)}
        if isinstance(y, pd.Series):
            y = np.array(y.map(label_mapping), dtype=int)
        else:
            y_series = pd.Series(y)
            y = np.array(y_series.map(label_mapping), dtype=int)
        
        if len(y) == 0 or len(np.unique(y)) < 2:
            raise ValueError(f"Недостаточно классов в y: {np.unique(y)}")
        
        if verbose:
            print(f"[INFO] Уникальные классы в y: {np.unique(y)}")
            print(f"[INFO] Распределение классов: {pd.Series(y).value_counts().to_dict()}")
        
        # ========== Стратифицированное разделение ==========
        if verbose:
            print("\n[INFO] ========== Стратифицированное разделение на train/test ==========")
        
        # Сначала разделяем индексы, чтобы потом использовать их для формирования датафрейма
        indices = np.arange(len(X))
        train_indices, test_indices = train_test_split(
            indices,
            test_size=test_size,
            random_state=random_state,
            stratify=y
        )
        
        # Теперь разделяем данные используя индексы
        if isinstance(X, pd.DataFrame):
            X_train = X.iloc[train_indices].reset_index(drop=True)
            X_test = X.iloc[test_indices].reset_index(drop=True)
        else:
            X_train = X[train_indices]
            X_test = X[test_indices]
        y_train = y[train_indices]
        y_test = y[test_indices]
        
        if verbose:
            print(f"[INFO] Train размер: {len(X_train)}, Test размер: {len(X_test)}")
            print(f"[INFO] Распределение классов в train: {pd.Series(y_train).value_counts().to_dict()}")
            print(f"[INFO] Распределение классов в test: {pd.Series(y_test).value_counts().to_dict()}")
        
        # Обучение на train
        automl_stratified = AutoML(
            task='classification',
            use_preprocessing_pipeline=True,
            feature_selector_type=None,
            use_val_test_pipeline=False,
            auto_models_init_kwargs={
                "metric": "roc_auc",
                "time_series": False,
                "models_list": ["linear", "forests", "boostings"],
                "blend": True,
                "stack": True,
                "n_splits": n_folds,
            },
            n_jobs=n_jobs,
            random_state=random_state,
        )
        
        if verbose:
            print("[INFO] Начало обучения модели на train данных...")
        
        automl_stratified = automl_stratified.fit(
            X_train, y_train,
            auto_model_fit_kwargs={"tuning_timeout": tuning_timeout}
        )
        
        if automl_stratified is None or not hasattr(automl_stratified, 'auto_model'):
            raise ValueError("AutoML (стратифицированный) не обучен корректно")
        
        # Предсказания на train и test
        predictions_train = automl_stratified.predict(X_train)
        predictions_test = automl_stratified.predict(X_test)
        
        # Обработка предсказаний
        if len(predictions_train.shape) == 2 and predictions_train.shape[1] >= 2:
            predictions_train_proba = predictions_train[:, 1]
        else:
            predictions_train_proba = predictions_train
        
        if len(predictions_test.shape) == 2 and predictions_test.shape[1] >= 2:
            predictions_test_proba = predictions_test[:, 1]
        else:
            predictions_test_proba = predictions_test
        
        # Метрики
        oof_score = automl_stratified.auto_model.best_score if hasattr(automl_stratified.auto_model, 'best_score') else None
        train_score = roc_auc_score(y_train, predictions_train_proba) if predictions_train_proba is not None else None
        test_score = roc_auc_score(y_test, predictions_test_proba) if predictions_test_proba is not None else None
        
        if verbose:
            print(f"[INFO] OOF score (train): {oof_score}")
            print(f"[INFO] ROC-AUC (train): {train_score}")
            print(f"[INFO] ROC-AUC (test): {test_score}")
        
        # ========== Формирование датафрейма с результатами ==========
        n_total = len(X)
        
        # Создаем датафрейм с правильной структурой
        df_results = pd.DataFrame(index=range(n_total))
        
        # Истинные значения для всех данных
        df_results['y_true'] = y
        df_results['is_test'] = False
        
        # Заполняем данные используя сохраненные индексы
        df_results.loc[train_indices, 'predictions_train'] = predictions_train_proba
        df_results.loc[test_indices, 'predictions_test'] = predictions_test_proba
        df_results.loc[train_indices, 'y_true_train'] = y_train
        df_results.loc[test_indices, 'y_true_test'] = y_test
        df_results.loc[test_indices, 'is_test'] = True
        
        # Добавляем метрики как отдельные колонки (одинаковые для всех строк)
        df_results['oof_score'] = oof_score
        df_results['train_score'] = train_score
        df_results['test_score'] = test_score
        df_results['task_id'] = task_id
        df_results['dataset_name'] = dataset.name
        
        if verbose:
            print(f"\n[INFO] Датафрейм создан. Размер: {df_results.shape}")
            print(f"[INFO] Колонки: {df_results.columns.tolist()}")
        
        return df_results
        
    except Exception as e:
        error_msg = f"Ошибка при выполнении пайплайна: {type(e).__name__}: {e}"
        if verbose:
            print(f"[ERROR] {error_msg}")
            print(f"[ERROR] Трассировка: {traceback.format_exc()}")
        
        # Возвращаем пустой датафрейм с правильной структурой
        return pd.DataFrame({
            'y_true': [],
            'is_test': [],
            'predictions_train': [],
            'predictions_test': [],
            'y_true_train': [],
            'y_true_test': [],
            'oof_score': [],
            'train_score': [],
            'test_score': [],
            'task_id': [],
            'dataset_name': [],
        })


## Пакетная обработка task_id из bench_merged.csv


In [8]:
def process_benchmark_csv(
    input_csv_path: str = "bench_merged.csv",
    output_csv_path: str = "bench_with_metrics.csv",
    sort_by: str = "num_cells",
    ascending: bool = True,
    test_size: float = 0.2,
    random_state: int = 42,
    n_jobs: int = 16,
    tuning_timeout: int = 10,
    verbose: bool = True
) -> pd.DataFrame:
    """
    Обрабатывает все task_id из bench_merged.csv, вычисляет метрики и сохраняет результаты.
    
    Параметры:
    ----------
    input_csv_path : str, default="bench_merged.csv"
        Путь к входному CSV файлу с task_id
    output_csv_path : str, default="bench_with_metrics.csv"
        Путь к выходному CSV файлу с результатами
    sort_by : str, default="num_cells"
        Колонка для сортировки task_id
    ascending : bool, default=True
        Сортировать по возрастанию
    test_size : float, default=0.2
        Доля данных для тестового набора
    random_state : int, default=42
        Random state для воспроизводимости
    n_jobs : int, default=16
        Количество параллельных процессов
    tuning_timeout : int, default=10
        Таймаут для тюнинга модели (в секундах)
    verbose : bool, default=True
        Выводить ли информационные сообщения
    
    Возвращает:
    -----------
    pd.DataFrame
        Датафрейм с результатами обработки всех task_id
    """
    import traceback
    from pathlib import Path
    
    # Метрики, которые должны быть заполнены
    metric_columns = ['oof_score', 'train_score', 'test_score']
    
    # Шаг 1: Чтение и подготовка данных
    if verbose:
        print(f"[INFO] Чтение файла {input_csv_path}...")
    
    if not Path(input_csv_path).exists():
        raise FileNotFoundError(f"Файл {input_csv_path} не найден")
    
    bench_df = pd.read_csv(input_csv_path)
    
    if 'task_id' not in bench_df.columns:
        raise ValueError(f"В файле {input_csv_path} отсутствует колонка 'task_id'")
    
    if verbose:
        print(f"[INFO] Загружено {len(bench_df)} строк из {input_csv_path}")
    
    # Извлечение и сортировка task_id
    if sort_by in bench_df.columns:
        bench_df = bench_df.sort_values(by=sort_by, ascending=ascending)
        if verbose:
            print(f"[INFO] Данные отсортированы по колонке '{sort_by}' ({'по возрастанию' if ascending else 'по убыванию'})")
    else:
        if verbose:
            print(f"[WARNING] Колонка '{sort_by}' не найдена, сортировка не выполнена")
    
    task_ids = bench_df['task_id'].unique().tolist()
    
    if verbose:
        print(f"[INFO] Найдено {len(task_ids)} уникальных task_id")
    
    # Шаг 4: Загрузка существующих результатов (если есть)
    if Path(output_csv_path).exists():
        if verbose:
            print(f"[INFO] Загрузка существующих результатов из {output_csv_path}...")
        results_df = pd.read_csv(output_csv_path)
        
        # Проверяем, что все исходные колонки присутствуют
        missing_cols = set(bench_df.columns) - set(results_df.columns)
        if missing_cols:
            if verbose:
                print(f"[WARNING] В результатах отсутствуют колонки: {missing_cols}. Они будут добавлены.")
            # Добавляем недостающие колонки
            for col in missing_cols:
                results_df[col] = None
    else:
        if verbose:
            print(f"[INFO] Файл {output_csv_path} не существует, будет создан новый")
        results_df = pd.DataFrame()
    
    # Убеждаемся, что колонки с метриками присутствуют
    for col in metric_columns:
        if col not in results_df.columns:
            results_df[col] = None
    
    # Шаг 2: Обработка каждого task_id
    processed_count = 0
    skipped_count = 0
    error_count = 0
    
    for idx, task_id in enumerate(task_ids, 1):
        if verbose:
            print(f"\n{'='*80}")
            print(f"[INFO] Обработка task_id {task_id} ({idx}/{len(task_ids)})")
            print(f"{'='*80}")
        
        # Проверка, нужно ли обрабатывать этот task_id
        needs_processing = False
        
        if len(results_df) == 0:
            # Если результатов нет, нужно обработать
            needs_processing = True
        else:
            # Ищем строку с таким task_id
            existing_row = results_df[results_df['task_id'] == task_id]
            
            if len(existing_row) == 0:
                # Строки нет - нужно обработать
                needs_processing = True
                if verbose:
                    print(f"[INFO] task_id {task_id} не найден в результатах, требуется обработка")
            else:
                # Строка есть - проверяем метрики
                row_metrics = existing_row[metric_columns].iloc[0]
                
                # Проверяем, заполнены ли все метрики (не NaN и не None)
                metrics_filled = all(
                    pd.notna(row_metrics[col]) and row_metrics[col] is not None 
                    for col in metric_columns
                )
                
                if not metrics_filled:
                    needs_processing = True
                    if verbose:
                        print(f"[INFO] task_id {task_id} найден, но метрики не заполнены, требуется обработка")
                else:
                    if verbose:
                        print(f"[INFO] task_id {task_id} уже обработан, пропускаем")
                    skipped_count += 1
                    continue
        
        if needs_processing:
            try:
                # Запускаем расчёт метрик
                predictions_df = run_automl_pipeline(
                    task_id=task_id,
                    test_size=test_size,
                    random_state=random_state,
                    n_jobs=n_jobs,
                    tuning_timeout=tuning_timeout,
                    verbose=verbose
                )
                
                if len(predictions_df) == 0:
                    if verbose:
                        print(f"[WARNING] Не удалось получить результаты для task_id {task_id}")
                    error_count += 1
                    continue
                
                # Извлекаем метрики (они одинаковые для всех строк)
                metrics = {
                    'oof_score': predictions_df['oof_score'].iloc[0] if 'oof_score' in predictions_df.columns else None,
                    'train_score': predictions_df['train_score'].iloc[0] if 'train_score' in predictions_df.columns else None,
                    'test_score': predictions_df['test_score'].iloc[0] if 'test_score' in predictions_df.columns else None,
                }
                
                # Находим исходную строку для этого task_id
                original_row = bench_df[bench_df['task_id'] == task_id].iloc[0].copy()
                
                # Добавляем метрики к исходной строке
                for metric_name, metric_value in metrics.items():
                    original_row[metric_name] = metric_value
                
                # Обновляем или добавляем строку в результаты
                if len(results_df) > 0 and task_id in results_df['task_id'].values:
                    # Обновляем существующую строку
                    mask = results_df['task_id'] == task_id
                    results_df.loc[mask, metric_columns] = [
                        metrics.get('oof_score'),
                        metrics.get('train_score'),
                        metrics.get('test_score')
                    ]
                    # Также обновляем другие колонки из исходного файла
                    for col in bench_df.columns:
                        if col in original_row and col in results_df.columns:
                            results_df.loc[mask, col] = original_row[col]
                else:
                    # Добавляем новую строку
                    # Если results_df пустой, создаём его с правильными колонками
                    if len(results_df) == 0:
                        # Создаём DataFrame с колонками из bench_df + метрики
                        all_cols = list(bench_df.columns) + metric_columns
                        results_df = pd.DataFrame(columns=all_cols)
                    # Убеждаемся, что все колонки присутствуют
                    new_row = original_row.copy()
                    # Добавляем недостающие колонки из results_df
                    for col in results_df.columns:
                        if col not in new_row.index:
                            new_row[col] = None
                    # Преобразуем в DataFrame с правильным порядком колонок
                    new_row_df = pd.DataFrame([new_row], columns=results_df.columns)
                    results_df = pd.concat([results_df, new_row_df], ignore_index=True)
                
                # Шаг 5: Сохранение после каждого расчёта
                results_df.to_csv(output_csv_path, index=False)
                
                if verbose:
                    print(f"[INFO] Результаты для task_id {task_id} сохранены в {output_csv_path}")
                    oof_str = f"{metrics['oof_score']:.4f}" if metrics['oof_score'] is not None else "None"
                    train_str = f"{metrics['train_score']:.4f}" if metrics['train_score'] is not None else "None"
                    test_str = f"{metrics['test_score']:.4f}" if metrics['test_score'] is not None else "None"
                    print(f"[INFO] Метрики: OOF={oof_str}, Train={train_str}, Test={test_str}")
                
                processed_count += 1
                
            except Exception as e:
                error_msg = f"Ошибка при обработке task_id {task_id}: {type(e).__name__}: {e}"
                if verbose:
                    print(f"[ERROR] {error_msg}")
                    print(f"[ERROR] Трассировка: {traceback.format_exc()}")
                
                # Сохраняем информацию об ошибке в результатах
                if len(results_df) == 0 or task_id not in results_df['task_id'].values:
                    # Добавляем строку с ошибкой
                    original_row = bench_df[bench_df['task_id'] == task_id].iloc[0].copy()
                    for metric_name in metric_columns:
                        original_row[metric_name] = None
                    original_row['error'] = str(e)
                    # Если results_df пустой, создаём его с правильными колонками
                    if len(results_df) == 0:
                        # Создаём DataFrame с колонками из bench_df + метрики + error
                        all_cols = list(bench_df.columns) + metric_columns + ['error']
                        results_df = pd.DataFrame(columns=all_cols)
                    # Убеждаемся, что все колонки присутствуют
                    for col in results_df.columns:
                        if col not in original_row.index:
                            original_row[col] = None
                    # Преобразуем в DataFrame с правильным порядком колонок
                    new_row_df = pd.DataFrame([original_row], columns=results_df.columns)
                    results_df = pd.concat([results_df, new_row_df], ignore_index=True)
                else:
                    # Обновляем строку с ошибкой
                    if 'error' not in results_df.columns:
                        results_df['error'] = None
                    mask = results_df['task_id'] == task_id
                    results_df.loc[mask, 'error'] = str(e)
                    # Обнуляем метрики для строки с ошибкой
                    results_df.loc[mask, metric_columns] = [None, None, None]
                
                # Сохраняем даже при ошибке
                results_df.to_csv(output_csv_path, index=False)
                
                error_count += 1
    
    # Итоговая статистика
    if verbose:
        print(f"\n{'='*80}")
        print(f"[INFO] Обработка завершена!")
        print(f"[INFO] Обработано: {processed_count}")
        print(f"[INFO] Пропущено (уже обработано): {skipped_count}")
        print(f"[INFO] Ошибок: {error_count}")
        print(f"[INFO] Всего task_id: {len(task_ids)}")
        print(f"[INFO] Результаты сохранены в {output_csv_path}")
        print(f"{'='*80}")
    
    return results_df


In [ ]:
# Пример использования функции для пакетной обработки
# Раскомментируйте и запустите для обработки всех task_id из bench_merged.csv

results_bench = process_benchmark_csv(
    input_csv_path="bench_merged.csv",
    output_csv_path="bench_with_metrics.csv",
    sort_by="num_cells",
    ascending=True,
    test_size=0.2,
    random_state=42,
    n_jobs=16,
    tuning_timeout=60,
    verbose=True
)

# Просмотр результатов
print("\n[INFO] Первые строки результатов:")
print(results_bench.head())

print("\n[INFO] Статистика по метрикам:")
print(results_bench[['task_id', 'oof_score', 'train_score', 'test_score']].describe())


[INFO] Чтение файла bench_merged.csv...
[INFO] Загружено 41 строк из bench_merged.csv
[INFO] Данные отсортированы по колонке 'num_cells' (по возрастанию)
[INFO] Найдено 41 уникальных task_id
[INFO] Загрузка существующих результатов из bench_with_metrics.csv...

[INFO] Обработка task_id 359955 (1/41)
[INFO] task_id 359955 уже обработан, пропускаем

[INFO] Обработка task_id 146818 (2/41)
[INFO] task_id 146818 не найден в результатах, требуется обработка
[INFO] Начало загрузки данных из OpenML (task_id=146818)...
[INFO] Задача загружена: 146818
[INFO] Датасет загружен: Australian
[INFO] Количество фолдов: 9
[INFO] Классы: ['0', '1']
[INFO] Данные загружены. Размер X: (690, 14), размер y: (690,)
[INFO] Уникальные классы в y: [0 1]
[INFO] Распределение классов: {0: 383, 1: 307}

[INFO] ========== Стратифицированное разделение на train/test ==========
[INFO] Train размер: 552, Test размер: 138
[INFO] Распределение классов в train: {0: 306, 1: 246}
[INFO] Распределение классов в test: {0: 77,

In [ ]:
# Пример использования функции
results_df = run_automl_pipeline(
    task_id=359955,
    test_size=0.2,
    random_state=42,
    n_jobs=16,
    tuning_timeout=10,
    verbose=True
)

# Просмотр результатов
print("\n[INFO] Первые строки датафрейма:")
print(results_df.head(10))

print("\n[INFO] Информация о датафрейме:")
print(results_df.info())

print("\n[INFO] Статистика метрик:")
metrics_cols = ['oof_score', 'train_score', 'test_score']
print(results_df[metrics_cols].iloc[0])

print("\n[INFO] Распределение по train/test:")
print(results_df['is_test'].value_counts())


[INFO] Начало загрузки данных из OpenML (task_id=359955)...
[INFO] Задача загружена: 359955
[INFO] Датасет загружен: blood-transfusion-service-center
[INFO] Количество фолдов: 9
[INFO] Классы: ['1', '2']
[INFO] Данные загружены. Размер X: (748, 4), размер y: (748,)
[INFO] Уникальные классы в y: [0 1]
[INFO] Распределение классов: {0: 570, 1: 178}

[INFO] ========== Стратифицированное разделение на train/test ==========
[INFO] Train размер: 598, Test размер: 150
[INFO] Распределение классов в train: {0: 456, 1: 142}
[INFO] Распределение классов в test: {0: 114, 1: 36}
[2025-11-14 20:59:25,237] - [  PREPROC   ] - Успешно заданы шаги pipeline
[INFO] Начало обучения модели на train данных...
[2025-11-14 20:59:25,241] - [Pipeline] .. (step 1 of 6) Processing nan_cols_dropper, total=   0.0
[2025-11-14 20:59:25,246] - [Pipeline] ....... (step 2 of 6) Processing nan_imputer, total=   0.0
[2025-11-14 20:59:25,249] - [Pipeline] .... (step 3 of 6) Processing qconst_dropper, total=   0.0
[2025-11-